# ETL Youtube Charts Brasil

#### Neste notebook, irei apresentar a forma de como extraí os dados dos charts do youtube, desde Janeiro de 2019 até Agosto de 2023. Usarei esses dados para analisar situações como, artistas com mais lançamentos, melhores datas de lançamento, tempo que tal música esteve nas paradas, entre outros assuntos. 
#### Você pode acessar os resultados desses dados no link abaixo:
####

-------------------------------------------------------------------------------------------------------------------------------

#### Bibliotecas que irei utilizar:

In [54]:
import datetime
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import requests


-------------------------------------------------------------------------------------------------------------------------------

#### Criação de Listagem com url's

Aqui, irei criar uma lista com as url's de cada chart que vou acessar para extrair as informações. O foco é usar um código que substituia os intervalos de data da url padrão.

In [ ]:
# Defini data de início
data1 = datetime.datetime.strptime("20190104", "%Y%m%d")
data2 = datetime.datetime.strptime("20190110", "%Y%m%d")

# Defini data limite
limite_data1 = datetime.datetime.strptime("20230811", "%Y%m%d")
limite_data2 = datetime.datetime.strptime("20230817", "%Y%m%d")

# Criar lista
urls = []

# Nomear a url que receberá as datas
base_url = "https://charts.youtube.com/charts/TopVideos/br/{}-{}"

# Loop para somar 7 dias em cada variável de data até a data limite e adiciona essa data em uma lista.
while data1 <= limite_data1 and data2 <= limite_data2:
    
    urls.append(base_url.format(data1.strftime("%Y%m%d"), data2.strftime("%Y%m%d")))
    
    data1 += datetime.timedelta(days=7)
    data2 += datetime.timedelta(days=7)

# Imprimi a lista de URLs
for url in urls:
    print(url)


-------------------------------------------------------------------------------------------------------------------------------

#### Webscrapping

Utilizando a lista de url's que criei anteriormente no webscrapping para extrair as informações e adiciona-las dentro de uma tabela.

In [45]:
# Configurando o Selenium
chrome_options = Options()

# Caminho para o chromedriver
driver_path = r"C:\Users\vinyc\Documents\teste_especialista\chromedriver-win64\chromedriver.exe"

# Inicializar o navegador usando o driver do Chrome
driver = webdriver.Chrome(executable_path=driver_path, options=chrome_options)
driver.maximize_window()

# Criar um dataframe vazio
df = pd.DataFrame(columns=["intervalo_data", "classificacao","semanas_chart","musica",
                        "artista","views_chart", "video_id"])

# Loop para rodar as urls com as datas
for data in urls:
    url = data  

    # Fazer o Selenium acessar a página
    driver.get(url)
    
    # espera até 20 segundos até aparecer todas a informações necessárias
    wait = WebDriverWait(driver, 20)  
    element = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='chart-table']/div/div")))
    
    # Encontrar a div com a classe especificada
    div_class_name = "//*[@id='chart-table']/div/div"
    chart_table_div = driver.find_element(By.XPATH, div_class_name)

    # Extrair o conteúdo HTML
    html_content = chart_table_div.get_attribute("innerHTML")

    # Configurando o beatifulsoup para usar no html
    soup = BeautifulSoup(html_content, "html.parser")
    
    # Separar a numeração da URL
    intervalo_data = url.split("/br/")[1].split("?hl=pt")[0]

    # Encontrar as classificações
    index_cells = soup.find_all('div', class_='index-cell style-scope ytmc-chart-table')

    # Encontrar as semanas nas paradas
    chart_period_cells = soup.find_all('div', class_='chart-period-cell style-scope ytmc-chart-table')

    # Encontrar o link do youtube para usar futuramente como video_id
    thumbnail_cells = soup.find_all('div', class_='thumbnail-cell style-scope ytmc-chart-table')
    
    #Encontrar o título da música
    music_title = soup.find_all('span', class_='ytmc-ellipsis-text style-scope')
    
    #Encontrar o nome do artista
    artist = soup.find_all('span', class_= 'ytmc-artist-name clickable style-scope ytmc-artists-list')

    #Encontrar as views do chart
    views_chart = soup.find_all('span', class_= 'views-cell style-scope ytmc-chart-table')

    # Iterar sobre as divs encontradas para extrair os dados
    for index, index_cell in enumerate(index_cells):
        # Extrair o valor do índice
        classificacao = index_cell.find('div', class_='rank style-scope ytmc-chart-table')
        
        music_title = index_cell.find_next('span', class_='ytmc-ellipsis-text style-scope').text.strip()
        
        #Extraindo os nomes dos artistas, como haviam nomes que não estavam sendo puxados, adicionei um try para não parar a ação
        try:
            artist = index_cell.find_next('span', class_='ytmc-artist-name clickable style-scope ytmc-artists-list')
            artist = artist.text if artist is not None else 'N/A'
        except AttributeError:
            artist = 'N/A'
            
        #Extraindo a parte de texto das views
        views_chart = index_cell.find_next('div', class_='views-cell style-scope ytmc-chart-table')
        views_chart.text

        # Extrair o valor do período do gráfico
        semanas_chart = chart_period_cells[index].find('span', class_='style-scope ytmc-chart-table')

        # Extrair o valor do video_id
        thumbnail_div = thumbnail_cells[index].find('img', class_='chart-entry-thumbnail clickable style-scope ytmc-chart-table')
        video_id_value = thumbnail_div.get('endpoint', '') if thumbnail_div else ''

        # Adicionar os dados ao DataFrame
        df = df.append({"intervalo_data": intervalo_data, "classificacao": classificacao, 
                        "semanas_chart": semanas_chart,"musica": music_title,
                        "artista": artist,"views_chart": views_chart, "video_id": video_id_value}, ignore_index=True)
        
# Fechar o navegador
driver.quit()

# Imprimir o DataFrame
df

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
0,20190104-20190110,None,None,Jenifer,Gabriel Diniz,[\n Visualizações\n ],
1,20190104-20190110,[1],[13 semanas],Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
2,20190104-20190110,[2],[8 semanas],Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
3,20190104-20190110,[3],[5 semanas],Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
4,20190104-20190110,[4],[16 semanas],Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
...,...,...,...,...,...,...,...
24336,20230811-20230817,[96],[1 semana],Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24337,20230811-20230817,[97],[6 semanas],Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24338,20230811-20230817,[98],[13 semanas],Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24339,20230811-20230817,[99],[26 semanas],Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."


-------------------------------------------------------------------------------------------------------------------------------

#### Criando um back up para os dados extraidos

In [82]:
df1 = df.copy()
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
0,20190104-20190110,None,None,Jenifer,Gabriel Diniz,[\n Visualizações\n ],
1,20190104-20190110,[1],[13 semanas],Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
2,20190104-20190110,[2],[8 semanas],Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
3,20190104-20190110,[3],[5 semanas],Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
4,20190104-20190110,[4],[16 semanas],Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
...,...,...,...,...,...,...,...
24336,20230811-20230817,[96],[1 semana],Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24337,20230811-20230817,[97],[6 semanas],Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24338,20230811-20230817,[98],[13 semanas],Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."
24339,20230811-20230817,[99],[26 semanas],Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]","{""urlEndpoint"":{""url"":""https://www.youtube.com..."


-------------------------------------------------------------------------------------------------------------------------------

#### Acessando a API do youtube para extrair mais informações

Extrair data de lançamento com api do yt, irei extrair somente os dados de data de lançamento e pra isso, vou isolar a coluna video_id para diminuir a quantidade de dados a ser retirado, considerando que a API do youtube tem uma limitação de 10mil requests por dia, assim, consiguirei todos os dados que preciso de forma mais rápida.

In [83]:
# Primeiro preciso limpar os dados vazios.
df1.dropna(inplace=True)


In [84]:
# Limpando a coluna video_id para deixar somente os id's necessários.
df1['video_id'] = df1['video_id'].apply(lambda x: x.split("v=")[-1].split('"')[0])

df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,[1],[13 semanas],Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]",D2rG7pXd2LY
2,20190104-20190110,[2],[8 semanas],Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]",1ppPuobqt-g
3,20190104-20190110,[3],[5 semanas],Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]",1MTsqRU-4LY
4,20190104-20190110,[4],[16 semanas],Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]",M76qUQTt_Sw
5,20190104-20190110,[5],[28 semanas],Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,"[\n, [\n, [9.57M], \n], \n]",rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,[96],[1 semana],Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]",CWk7e39jPFs
24337,20230811-20230817,[97],[6 semanas],Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]",0dLX40UMUKo
24338,20230811-20230817,[98],[13 semanas],Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]",7EjIdjKNRls
24339,20230811-20230817,[99],[26 semanas],Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]",xEd4vHYLDko


In [49]:
# Criar um novo df para receber somente os id's distintos.
id_view = df1[['video_id']].drop_duplicates().reset_index(drop=True)
id_view

,video_id
0,
1,D2rG7pXd2LY
2,1ppPuobqt-g
3,1MTsqRU-4LY
4,M76qUQTt_Sw
...,...
2560,_TGgfe5RrdE
2561,ypiw-p0XaJU
2562,2rP-Kp-dqFQ
2563,-JBsQjMynbM


In [56]:
#código para acessar api e retirar a informação específica.
def get_youtube_video(video_id, api_key):
    base_url = "https://www.googleapis.com/youtube/v3/videos"
    params = {
        "key": api_key,
        "id": video_id,
        "part": "snippet"
    }

    response = requests.get(base_url, params=params)
    data = response.json()
    
    # Nessa parte, é onde eu peço o item publishedAt para a Api me retornar a data de lançamento.
    if "items" in data and len(data["items"]) > 0:
        video_info = data["items"][0]
        published_at = video_info["snippet"]["publishedAt"]
        return published_at

# Ocultei a api_key por motivos de segurança.
if __name__ == "__main__":
    api_key = "*********************"
    
    # Aplico a função em uma nova coluna.
    id_view['data_lancamento'] = id_view['video_id'].apply(get_youtube_video, args=(api_key,))
    
    id_view.set_index('video_id', inplace=True)

id_view 

,data_lancamento
video_id,
,None
D2rG7pXd2LY,2018-09-21T15:00:02Z
1ppPuobqt-g,2018-11-15T21:00:00Z
1MTsqRU-4LY,2018-12-07T18:00:29Z
M76qUQTt_Sw,2018-09-20T16:01:37Z
...,...
_TGgfe5RrdE,2023-08-09T15:00:07Z
ypiw-p0XaJU,2023-08-11T14:00:11Z
2rP-Kp-dqFQ,2023-05-08T00:15:10Z


-------------------------------------------------------------------------------------------------------------------------------

#### Tratamento dos dados

Agora começo a parte mais trabalhosa, que é a limpeza dos dados extraidos, e temos muitos dados que foram retirados de HTML, então temos que ficar atentos sobre as trocas de formatos. Eu tenho como preferencia formata-los para strings e depois realizar as ações específicas para extrair a informação que eu quero. 

In [ ]:
# Transformação do tipo da coluna classificacao.
df1['classificacao'] = df['classificacao'].astype(str)

In [ ]:
# Extração da informação necessária utilizando replace em um padrão de caracteres.
df1['classificacao'] = df1['classificacao'].str.replace('[^\d]', '')

In [87]:
# Nesse caso, logo quando transformamos a coluna, podemos observar que ela contém resíduos do código html ao qual estava vinculada. 
df1['semanas_chart'] = df['semanas_chart'].astype(str)
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,"<span class=""style-scope ytmc-chart-table"">13 ...",Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]",D2rG7pXd2LY
2,20190104-20190110,2,"<span class=""style-scope ytmc-chart-table"">8 s...",Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]",1ppPuobqt-g
3,20190104-20190110,3,"<span class=""style-scope ytmc-chart-table"">5 s...",Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]",1MTsqRU-4LY
4,20190104-20190110,4,"<span class=""style-scope ytmc-chart-table"">16 ...",Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]",M76qUQTt_Sw
5,20190104-20190110,5,"<span class=""style-scope ytmc-chart-table"">28 ...",Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,"[\n, [\n, [9.57M], \n], \n]",rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,"<span class=""style-scope ytmc-chart-table"">1 s...",Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]",CWk7e39jPFs
24337,20230811-20230817,97,"<span class=""style-scope ytmc-chart-table"">6 s...",Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]",0dLX40UMUKo
24338,20230811-20230817,98,"<span class=""style-scope ytmc-chart-table"">13 ...",Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]",7EjIdjKNRls
24339,20230811-20230817,99,"<span class=""style-scope ytmc-chart-table"">26 ...",Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]",xEd4vHYLDko


In [88]:
# Para extrair os dados dessa coluna, irei utilizar o BeatifulSoup para me dar somente o texto contido.
df1['semanas_chart'] = df1['semanas_chart'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,13 semanas,Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]",D2rG7pXd2LY
2,20190104-20190110,2,8 semanas,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]",1ppPuobqt-g
3,20190104-20190110,3,5 semanas,Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]",1MTsqRU-4LY
4,20190104-20190110,4,16 semanas,Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]",M76qUQTt_Sw
5,20190104-20190110,5,28 semanas,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,"[\n, [\n, [9.57M], \n], \n]",rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1 semana,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]",CWk7e39jPFs
24337,20230811-20230817,97,6 semanas,Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]",0dLX40UMUKo
24338,20230811-20230817,98,13 semanas,Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]",7EjIdjKNRls
24339,20230811-20230817,99,26 semanas,Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]",xEd4vHYLDko


In [89]:
# E agora, separar a palavra semana ou semanas e deixar somente os números.
df1['semanas_chart'] = df1['semanas_chart'].str.replace('semanas', '').str.replace('semana', '')
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,13,Jenifer,Gabriel Diniz,"[\n, [\n, [18.7M], \n], \n]",D2rG7pXd2LY
2,20190104-20190110,2,8,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,"[\n, [\n, [11.5M], \n], \n]",1ppPuobqt-g
3,20190104-20190110,3,5,Agora é Tudo Meu,MC Kevinho,"[\n, [\n, [9.9M], \n], \n]",1MTsqRU-4LY
4,20190104-20190110,4,16,Atrasadinha (Ao Vivo),Felipe Araújo,"[\n, [\n, [9.63M], \n], \n]",M76qUQTt_Sw
5,20190104-20190110,5,28,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,"[\n, [\n, [9.57M], \n], \n]",rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,"[\n, [\n, [992K], \n], \n]",CWk7e39jPFs
24337,20230811-20230817,97,6,Céu Azul,Charlie Brown Jr.,"[\n, [\n, [974K], \n], \n]",0dLX40UMUKo
24338,20230811-20230817,98,13,Sinônimos,Chitãozinho & Xororó,"[\n, [\n, [966K], \n], \n]",7EjIdjKNRls
24339,20230811-20230817,99,26,Perdoou Nada (Let's Bora),Israel & Rodolffo,"[\n, [\n, [941K], \n], \n]",xEd4vHYLDko


In [ ]:
# Na coluna views_chart, irei repetir o mesmo processo do semanas_chart
df1['views_chart'] = df['views_chart'].astype(str)

In [91]:
# Só que dessa vez, a coluna contém mais informação inseridas como texto.
df1['views_chart'] = df1['views_chart'].apply(lambda x: BeautifulSoup(x, 'html.parser').get_text())
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,13,Jenifer,Gabriel Diniz,\n\n18.7M\n\n,D2rG7pXd2LY
2,20190104-20190110,2,8,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,\n\n11.5M\n\n,1ppPuobqt-g
3,20190104-20190110,3,5,Agora é Tudo Meu,MC Kevinho,\n\n9.9M\n\n,1MTsqRU-4LY
4,20190104-20190110,4,16,Atrasadinha (Ao Vivo),Felipe Araújo,\n\n9.63M\n\n,M76qUQTt_Sw
5,20190104-20190110,5,28,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,\n\n9.57M\n\n,rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,\n\n992K\n\n,CWk7e39jPFs
24337,20230811-20230817,97,6,Céu Azul,Charlie Brown Jr.,\n\n974K\n\n,0dLX40UMUKo
24338,20230811-20230817,98,13,Sinônimos,Chitãozinho & Xororó,\n\n966K\n\n,7EjIdjKNRls
24339,20230811-20230817,99,26,Perdoou Nada (Let's Bora),Israel & Rodolffo,\n\n941K\n\n,xEd4vHYLDko


In [96]:
# Vamos limpa-las
df1['views_chart'] = df1['views_chart'].str.replace('\n', '')
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,13,Jenifer,Gabriel Diniz,18.7M,D2rG7pXd2LY
2,20190104-20190110,2,8,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,11.5M,1ppPuobqt-g
3,20190104-20190110,3,5,Agora é Tudo Meu,MC Kevinho,9.9M,1MTsqRU-4LY
4,20190104-20190110,4,16,Atrasadinha (Ao Vivo),Felipe Araújo,9.63M,M76qUQTt_Sw
5,20190104-20190110,5,28,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,9.57M,rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,992K,CWk7e39jPFs
24337,20230811-20230817,97,6,Céu Azul,Charlie Brown Jr.,974K,0dLX40UMUKo
24338,20230811-20230817,98,13,Sinônimos,Chitãozinho & Xororó,966K,7EjIdjKNRls
24339,20230811-20230817,99,26,Perdoou Nada (Let's Bora),Israel & Rodolffo,941K,xEd4vHYLDko


In [94]:
# Para manter o formato de Milhores e Milhares da coluna, irei aplicar essa função, substituindo as letras a seguir dos números
# em zeros, correspondentes a numeração.
def convert_views(value):
    if 'M' in value:
        return float(value.replace('M', '')) * 1e6
    elif 'K' in value:
        return float(value.replace('K', '')) * 1e3
    else:
        return float(value)

In [97]:
# Aplicando a função.
df1['views_chart'] = df1['views_chart'].apply(convert_views)
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id
1,20190104-20190110,1,13,Jenifer,Gabriel Diniz,18700000.0,D2rG7pXd2LY
2,20190104-20190110,2,8,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,11500000.0,1ppPuobqt-g
3,20190104-20190110,3,5,Agora é Tudo Meu,MC Kevinho,9900000.0,1MTsqRU-4LY
4,20190104-20190110,4,16,Atrasadinha (Ao Vivo),Felipe Araújo,9630000.0,M76qUQTt_Sw
5,20190104-20190110,5,28,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,9570000.0,rYKOuKaWEjg
...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,992000.0,CWk7e39jPFs
24337,20230811-20230817,97,6,Céu Azul,Charlie Brown Jr.,974000.0,0dLX40UMUKo
24338,20230811-20230817,98,13,Sinônimos,Chitãozinho & Xororó,966000.0,7EjIdjKNRls
24339,20230811-20230817,99,26,Perdoou Nada (Let's Bora),Israel & Rodolffo,941000.0,xEd4vHYLDko


In [102]:
# Finalizando a limpeza, irei adicionar uma coluna com uma condição, somente para utilizarmos em uma análise futura.

def categorias(semanas):
    if semanas <= 4:
        return '1 mês'
    elif 5 <= semanas <= 12:
        return '1 a 3 meses'
    elif 13 <= semanas <= 24:
        return '3 a 6 meses'
    else:
        return '6 meses ou mais'

# Converte a coluna para numérico
df1['semanas_chart'] = pd.to_numeric(df1['semanas_chart'], errors='coerce')

# Aplica a função
df1['categoria_meses'] = df1['semanas_chart'].apply(categorias)
df1

,intervalo_data,classificacao,semanas_chart,musica,artista,views_chart,video_id,categoria_meses
1,20190104-20190110,1,13,Jenifer,Gabriel Diniz,18700000.0,D2rG7pXd2LY,3 a 6 meses
2,20190104-20190110,2,8,Hoje Eu Vou Parar Na Gaiola,Mc Livinho,11500000.0,1ppPuobqt-g,1 a 3 meses
3,20190104-20190110,3,5,Agora é Tudo Meu,MC Kevinho,9900000.0,1MTsqRU-4LY,1 a 3 meses
4,20190104-20190110,4,16,Atrasadinha (Ao Vivo),Felipe Araújo,9630000.0,M76qUQTt_Sw,3 a 6 meses
5,20190104-20190110,5,28,Notificação Preferida (Ao Vivo),Zé Neto & Cristiano,9570000.0,rYKOuKaWEjg,6 meses ou mais
...,...,...,...,...,...,...,...,...
24336,20230811-20230817,96,1,Ninguém Faz Igual (Ao Vivo),Rionegro & Solimões,992000.0,CWk7e39jPFs,1 mês
24337,20230811-20230817,97,6,Céu Azul,Charlie Brown Jr.,974000.0,0dLX40UMUKo,1 a 3 meses
24338,20230811-20230817,98,13,Sinônimos,Chitãozinho & Xororó,966000.0,7EjIdjKNRls,3 a 6 meses
24339,20230811-20230817,99,26,Perdoou Nada (Let's Bora),Israel & Rodolffo,941000.0,xEd4vHYLDko,6 meses ou mais


#### Finalizações

Agradeço a todos que chegaram até o final desse projeto, lembrando que é somente a parte de extração e transformação dos dados, abaixo, deixo meus links para contato, para sugestões, críticas, ideias e elogios:

[Medium](https://medium.com/@m.vinicius182)
[Linkedin](https://www.linkedin.com/in/m-vinicius-carvalho/)
[GitHub](https://github.com/mvinicius-182/m.vinicius)